## Start Spark session

In [ ]:
val config = new SparkConf().
set("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.0").
set("spark.jars.excludes", "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11")..
setAppName("EventGeneration")

val spark = new SparkContext(config)

In [2]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.0",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11"
    }
}

In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql._
import org.apache.spark.sql.types._

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1531125931988_0006,spark,idle,Link,Link,✔


SparkSession available as 'spark'.
import org.apache.spark.sql.types._

In [203]:
spark.sparkContext.stop

## Drop topics

In [ ]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --delete --topic rawevents --zookeeper "zk0-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk2-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk3-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --delete --topic streamPerHourPerDay --zookeeper "zk0-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk2-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk3-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --delete --topic streamPerHour --zookeeper "zk0-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk2-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk3-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --delete --topic streamPerHourAlert2 --zookeeper "zk0-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk2-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk3-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181"

## Create topics

In [251]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --replication-factor 3 --partitions 8 --topic rawevents --zookeeper "zk0-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk2-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk3-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --replication-factor 3 --partitions 8 --topic streamPerHourPerDay --zookeeper "zk0-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk2-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk3-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --replication-factor 3 --partitions 8 --topic streamPerHour --zookeeper "zk0-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk2-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk3-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --replication-factor 3 --partitions 8 --topic streamPerHourAlert2 --zookeeper "zk0-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk2-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk3-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181"

Created topic "rawevents".
Created topic "streamPerHourPerDay".
Created topic "streamPerHour".
Created topic "streamPerHourAlert2".


## Delete all checkpoints directory and output folders

In [252]:
%%bash
hdfs dfs -rm -r /path/checkpointstreamPerHour
hdfs dfs -rm -r /path/checkpointstreamPerHourPerDay
hdfs dfs -rm -r /path/chkpointalertType1query
hdfs dfs -rm -r /path/checkpointstreamPerHourAlert2
hdfs dfs -rm -r /path/chkpointalertType2query
hdfs dfs -rm -r /example/AlertType1
hdfs dfs -rm -r /example/AlertType2

rm: `/path/checkpointstreamPerHour': No such file or directory
rm: `/path/checkpointstreamPerHourPerDay': No such file or directory
rm: `/path/chkpointalertType1query': No such file or directory
rm: `/path/checkpointstreamPerHourAlert2': No such file or directory
rm: `/path/chkpointalertType2query': No such file or directory
rm: `/example/AlertType1': No such file or directory
rm: `/example/AlertType2': No such file or directory


## Create schemas

In [253]:
// The Kafka broker hosts and topic used to write to Kafka
val kafkaBrokers="wn0-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:9092,wn1-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:9092,wn2-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:9092"
val kafkaTopic="rawevents"
val zookeepers = "zk0-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk2-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181,zk3-kafka.wwpke2vrtm5e1fnmszmmcb3npe.cx.internal.cloudapp.net:2181"

// Define a schema for the data
val schema = (new StructType).
add("house_id", StringType).
add("timestamp", StringType).
add("value", StringType).
add("household_id", StringType)

var kafkastreamPerHourPerDaySchema = (new StructType).
add("house_id", StringType).
add("household_id", StringType).
add("Date", DateType).
add("LastHour", StringType).
add("Id", StringType).
add("TotalConsumptionForThatHourAndDay",DoubleType).
add("window",(new StructType).
    add("start",TimestampType).
    add("end",TimestampType)
   )

var kafkastreamPerHourSchema = (new StructType).
add("house_id", StringType).
add("household_id", StringType).
add("LastHour", StringType).
add("MeanConsumptionForThatHour",DoubleType).
add("SDConsumptionForThatHour",DoubleType).
add("1SDConsumption", DoubleType).
add("window",(new StructType).
    add("start",TimestampType).
    add("end",TimestampType)
   )

var kafkastreamPerHourSchemaAlert2 = (new StructType).
add("house_id", StringType).
add("LastHour", StringType).
add("MeanConsumptionForThatHour",DoubleType).
add("SDConsumptionForThatHour",DoubleType).
add("1SDConsumption", DoubleType).
add("window",(new StructType).
    add("start",TimestampType).
    add("end",TimestampType)
   )

kafkastreamPerHourSchemaAlert2: org.apache.spark.sql.types.StructType = StructType(StructField(house_id,StringType,true), StructField(LastHour,StringType,true), StructField(MeanConsumptionForThatHour,DoubleType,true), StructField(SDConsumptionForThatHour,DoubleType,true), StructField(1SDConsumption,DoubleType,true), StructField(window,StructType(StructField(start,TimestampType,true), StructField(end,TimestampType,true)),true))

## Create dataframes and queries

1. Create common dataframe on top of rawevents topic

In [254]:
var commonDF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", kafkaTopic).
option("startingOffsets","earliest").
option("failOnDataLoss","false").
load().
selectExpr("CAST(key AS STRING) AS key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), schema).alias("consumption")).
select("consumption.*").
selectExpr(
    "house_id",
    "household_id",
    "cast(cast(timestamp as int) as timestamp) as timestamp",
    "cast(value as double) as value",
    "hour(from_unixtime(cast(timestamp as int))) as LastHour",
    "cast(from_unixtime(cast(timestamp as int)) as date) as Date"
).
withColumn("Id",concat(
    col("house_id"),lit("_"),
    col("household_id"),lit("_"),
    date_format(col("Date"),"dd-MM-yyyy"),lit("_"),
    col("LastHour"))).
withWatermark("timestamp","1 minute").
dropDuplicates("Id")

commonDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [house_id: string, household_id: string ... 5 more fields]

## Start query to get hourly conumption for household for that hour for that day

In [255]:
var streamPerHourPerDay = commonDF.
groupBy(
    window($"timestamp", "1 hour", "1 hour"),
    col("house_id"),col("household_id"),col("Date"),col("LastHour"),col("Id")
).
agg(
    sum("value").as("TotalConsumptionForThatHourAndDay"),
    min("timestamp").as("MinTimestamp")
).
selectExpr("cast(house_id as string) as key","to_json(struct(*)) AS value","CAST(MinTimestamp AS string)AS timestamp").
as[(String,String,String)].
writeStream.
format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointstreamPerHourPerDay").
option("topic", "streamPerHourPerDay").
outputMode("append").
start()

streamPerHourPerDay: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@578dce78

In [ ]:
var streamPerHourPerDayTesting = commonDF.
groupBy(
    window($"timestamp", "1 hour", "1 hour"),
    col("house_id"),col("household_id"),col("Date"),col("LastHour"),col("Id")
).
agg(
    sum("value").as("TotalConsumptionForThatHourAndDay"),
    min("timestamp").as("MinTimestamp")
).
selectExpr("cast(house_id as string) as key","to_json(struct(*)) AS value","CAST(MinTimestamp AS string)AS timestamp").
as[(String,String,String)].
writeStream.
format("csv").
option("path","/example/streamPerHourPerDayTesting").
option("checkpointLocation", "/path/chkpointalertType2query").
option("failOnDataLoss","false").
outputMode("append").
start() 

In [268]:
streamPerHourPerDay.status

res19: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}

## Start query to get hourly conumption for a household for that hour from all past events

In [257]:
var streamPerHour = commonDF.
groupBy(
    window($"timestamp", "1 hour", "1 hour"),
    col("house_id"),col("household_id"),col("LastHour")
).
agg(
    mean("value").as("MeanConsumptionForThatHour")
    ,stddev_pop("value").as("SDConsumptionForThatHour")
    ,min("timestamp").as("MinTimestamp")
).
withColumn(
    "1SDConsumption"
    ,col("MeanConsumptionForThatHour") + col("SDConsumptionForThatHour")
).
selectExpr("cast(house_id as string) as key","to_json(struct(*)) AS value","CAST(MinTimestamp AS string)AS timestamp").
as[(String,String,String)].
writeStream.
format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointstreamPerHour").
option("topic", "streamPerHour").
outputMode("complete").
start()


streamPerHour: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6681a1e7

In [166]:
streamPerHour.status

res18: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}

## Start query to get hourly conumption for all households for that hour from all past events

In [258]:
var streamPerHourAlert2 = commonDF.
groupBy(
    window($"timestamp", "1 hour", "1 hour"),
    col("house_id"),col("LastHour")
).
agg(
    mean("value").as("MeanConsumptionForThatHour")
    ,stddev_pop("value").as("SDConsumptionForThatHour")
    ,min("timestamp").as("MinTimestamp")
).
withColumn(
    "1SDConsumption"
    ,col("MeanConsumptionForThatHour") + col("SDConsumptionForThatHour")
).
selectExpr("cast(house_id as string) as key","to_json(struct(*)) AS value","CAST(MinTimestamp AS string)AS timestamp").
as[(String,String,String)].
writeStream.
format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointstreamPerHourAlert2").
option("topic", "streamPerHourAlert2").
outputMode("complete").
start()

streamPerHourAlert2: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5f49a475

In [263]:
streamPerHourAlert2.status

res14: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}

## Create dataframes and join queries
1. Consume date from streamPerHourPerDay topic
2. Consume date from streamPerHour topic
3. Consume date from streamPerHourAlert2 topic
4. Prepare a join condition for 1st adn 2nd dataframes for alert type 1 
5. Prepare a join condition for 1st adn 3rd dataframes for alert type 2

In [259]:
var kafkastreamPerHourPerDayDF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", "streamPerHourPerDay").
option("startingOffsets","earliest").
option("failOnDataLoss","false").
load().
selectExpr("CAST(key AS STRING) AS key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), kafkastreamPerHourPerDaySchema).alias("streamPerHourPerDay")).
select("streamPerHourPerDay.*").
select(
    col("house_id"),
    col("household_id"),
    col("LastHour"),
    col("Id"),
    col("window.start").as("Start"),
    col("window.end").as("End"),
    col("TotalConsumptionForThatHourAndDay")
).
withWatermark("Start","5 minute")

var kafkastreamPerHourDF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", "streamPerHour").
option("startingOffsets","earliest").
option("failOnDataLoss","false").
load().
selectExpr("CAST(key AS STRING) AS key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), kafkastreamPerHourSchema).alias("streamPerHour")).
select("streamPerHour.*").
select(
    col("house_id"),
    col("household_id"),
    col("LastHour"),
    col("window.start").as("Start"),
    col("window.end").as("End"),
    col("1SDConsumption"),
    col("MeanConsumptionForThatHour"),
    col("SDConsumptionForThatHour")
).
withWatermark("Start","5 minute")

var kafkastreamPerHourAlert2DF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", "streamPerHourAlert2").
option("startingOffsets","earliest").
option("failOnDataLoss","false").
load().
selectExpr("CAST(key AS STRING) AS key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), kafkastreamPerHourSchemaAlert2).alias("streamPerHour")).
select("streamPerHour.*").
select(
    col("house_id"),
    col("LastHour"),
    col("window.start").as("Start"),
    col("window.end").as("End"),
    col("1SDConsumption"),
    col("MeanConsumptionForThatHour"),
    col("SDConsumptionForThatHour")
).
withWatermark("Start","5 minute")

var finalResultAlert1 = kafkastreamPerHourPerDayDF.as("x").join(
    kafkastreamPerHourDF.as("y"),
    expr("""
        x.house_id = y.house_id AND
        x.household_id = y.household_id AND
        x.LastHour = y.LastHour AND
        x.Start = y.Start AND x.End = y.End AND
        TotalConsumptionForThatHourAndDay > 1SDConsumption
        """),
    "leftOuter"
).
select(col("x.Id"),col("x.house_id"),col("x.household_id"),col("x.LastHour"),
       col("x.TotalConsumptionForThatHourAndDay"),
       col("y.1SDConsumption"),
       col("y.MeanConsumptionForThatHour"),
       col("y.SDConsumptionForThatHour")
      )

var finalResultAlert2 = kafkastreamPerHourPerDayDF.as("x").join(
    kafkastreamPerHourAlert2DF.as("y"),
    expr("""
        x.house_id = y.house_id AND
        x.LastHour = y.LastHour AND
        x.Start = y.Start AND x.End = y.End AND
        TotalConsumptionForThatHourAndDay > 1SDConsumption
        """),
    "leftOuter"
).
select(col("x.Id"),col("x.house_id"),col("x.LastHour"),
       col("x.TotalConsumptionForThatHourAndDay"),
       col("y.1SDConsumption"),
       col("y.MeanConsumptionForThatHour"),
       col("y.SDConsumptionForThatHour")
      )

finalResultAlert2: org.apache.spark.sql.DataFrame = [Id: string, house_id: string ... 5 more fields]

## Generate alert type 1

In [261]:
import org.apache.spark.sql.streaming.Trigger

var alertType1query = finalResultAlert1.
writeStream.
format("csv").
option("path","/example/AlertType1").
option("checkpointLocation", "/path/chkpointalertType1query").
option("failOnDataLoss","false").
outputMode("append").
start() 

alertType1query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@7aa0fdeb

In [206]:
alertType1query.status

res55: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Getting offsets from KafkaSource[Subscribe[streamPerHourPerDay]]",
  "isDataAvailable" : false,
  "isTriggerActive" : true
}

## Generate alert type 2

In [262]:
var alertType2query = finalResultAlert2.
writeStream.
format("csv").
option("path","/example/AlertType2").
option("checkpointLocation", "/path/chkpointalertType2query").
option("failOnDataLoss","false").
outputMode("append").
start() 

alertType2query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3f5720f

In [231]:
alertType2query.status

res19: Array[org.apache.spark.sql.streaming.StreamingQueryProgress] =
Array({
  "id" : "f2e94199-2d8e-4592-8fa4-f6ac5f61b643",
  "runId" : "42d73b2d-b997-4bc6-a179-2b38788d74f6",
  "name" : null,
  "timestamp" : "2018-07-08T13:39:51.178Z",
  "batchId" : 0,
  "numInputRows" : 0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "addBatch" : 234708,
    "getBatch" : 9,
    "getOffset" : 8279,
    "queryPlanning" : 151,
    "triggerExecution" : 243619,
    "walCommit" : 401
  },
  "eventTime" : {
    "watermark" : "1970-01-01T00:00:00.000Z"
  },
  "stateOperators" : [ {
    "numRowsTotal" : 0,
    "numRowsUpdated" : 0,
    "memoryUsedBytes" : 57399
  } ],
  "sources" : [ {
    "description" : "KafkaSource[Subscribe[streamPerHourPerDay]]",
    "startOffset" : null,
    "endOffset" : ...

In [241]:
alertType1query.stop
streamPerHourPerDay.stop
streamPerHour.stop
streamPerHourAlert2.stop
alertType2query.stop

In [74]:
%%bash
hdfs dfs -ls /example/AlertType2

Found 107 items
drwxr-xr-x   - livy supergroup          0 2018-07-07 20:31 /example/AlertType2/_spark_metadata
-rw-r--r--   1 livy supergroup          0 2018-07-07 20:08 /example/AlertType2/part-00000-13944819-93f1-4091-86f5-41c8a4b87c7b-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-07 19:59 /example/AlertType2/part-00000-15aff356-25d4-40b2-a741-07bbd2b256bb-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-07 19:50 /example/AlertType2/part-00000-1c525fbd-3a7b-4507-8559-48cd1da5e05d-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-07 20:05 /example/AlertType2/part-00000-337f1dbe-020e-49ce-83df-78af162cf0be-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-07 20:18 /example/AlertType2/part-00000-3ba1d0d9-e5f4-421c-bacf-5dcfd4b9e3bd-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-07 20:28 /example/AlertType2/part-00000-5b307dee-ca54-4e5f-8ca4-775e9b0c3e9a-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-07 20:30 /example/Alert

In [67]:
%%bash
hdfs dfs -cat /example/AlertType2/part-00000-c9b13960-8dbc-4176-8ed7-278c89b14ccc-c000.csv

2_0_1-9-2013_22,2,22,668.7830000000001,,,
2_0_2-9-2013_22,2,22,3840.0310000000013,,,


In [99]:
%%bash
hdfs dfs -getmerge /example/AlertType1 /home/sshuser/AlertType1.csv

getmerge: /home/sshuser/AlertType1.csv (Permission denied)
